# Imports

In [13]:
import rdrobust
import numpy as np
import pandas as pd

# Random Variables Generation 

In [109]:
sd = np.random.default_rng(seed=42)
print(sd)

Generator(PCG64)


In [110]:
n = 1000

X = np.random.uniform(-1,1,n)
epsilon = np.random.normal(0,0.5,n)

,X,epsilon
0,-0.376866,-0.240330
1,0.860636,0.235769
2,0.607797,-0.656938
3,0.377353,-0.135631
4,0.373133,-0.178618
...,...,...
995,0.997834,0.071953
996,0.522587,-0.015899
997,0.617594,-0.048474
998,-0.936216,-0.663257


In [111]:
L = np.array([0,10,20,30,40])
L = 0

In [113]:
def sign(x):
    if x<0:
        return -1
    else:
        return 1

def indicator(x):
    if np.abs(x)<0.1:
        return 1
    else:
        return 0
    
def mu(l,x):
    return sign(x)*np.power(x,2) - l*sign(x)*(np.power(x-0.1*sign(x),2)-np.power(0.1,2)*sign(x))*indicator(x)

In [134]:
Y = {}
for i in range(n):
    Y = np.append(Y,mu(L,X[i]) + epsilon[i])
Y = np.delete(Y,0)

In [135]:
len(Y)

1000

In [137]:
sample = pd.DataFrame({'X': X,'Y': Y})

In [138]:
sample

,X,Y
0,-0.248960,-0.340647
1,-0.297755,0.164998
2,-0.283357,0.937426
3,-0.123340,-0.232191
4,0.152658,-0.622129
...,...,...
995,0.618842,-0.763299
996,0.237602,0.181647
997,-0.306569,0.164812
998,0.075308,-0.557306
